<a href="https://colab.research.google.com/github/nakram21/Web-Scrapping/blob/main/Web_Scrapping_finhubNews2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#FINHUB_KEY='cecuohaad3i8too9m2v0cecuohaad3i8too9m2vg'

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 6.7 MB/s 
     |████████████████████████████████| 147 kB 14.9 MB/s 
     |████████████████████████████████| 178 kB 51.5 MB/s 
     |████████████████████████████████| 981 kB 52.1 MB/s 
     |████████████████████████████████| 5.4 MB 41.0 MB/s 
     |████████████████████████████████| 14.9 MB 47.7 MB/s 
     |████████████████████████████████| 9.7 MB 53.5 MB/s 
     |████████████████████████████████| 227 kB 73.9 MB/s 
     |████████████████████████████████| 510 kB 53.9 MB/s 
     |████████████████████████████████| 61 kB 8.1 MB/s 
     |████████████████████████████████| 137 kB 56.2 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993211 sha256=e8496ea65879a90e99f3c87b2a020f96a2e0eb6b6db1d692e78b02701f2dadcb
  Stored in directory: /root/.cache/pip/wheels/1e/80/23/0a24928ec3a3906ff5027f38d2fea824e7e97f2ba7c83d91e3

In [ ]:
!pip install python-decouple==3.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!/usr/local/bin/python3.7
# -*- coding: utf-8 -*-
###############################################################################
#
#  The MIT License (MIT)
#  Copyright (c) 2021 Philippe Ostiguy
#
#  Permission is hereby granted, free of charge, to any person obtaining a copy
#  of this software and associated documentation files (the "Software"), to deal
#  in the Software without restriction, including without limitation the rights
#  to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#  copies of the Software, and to permit persons to whom the Software is
#  furnished to do so, subject to the following conditions:
#
#  The above copyright notice and this permission notice shall be included in all
#  copies or substantial portions of the Software.
#
#  THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
#  EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
#  MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
#  IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM,
#  DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR
#  OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE
#  OR OTHER DEALINGS IN THE SOFTWARE.
###############################################################################

"""It's the module for web scraping. The module uses different sources and APIs for web-scraping"""

import os
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
import json
from decouple import config
from langdetect import detect
import time
import sqlite3
import bs4 as bs

def delta_date(start_date,end_date):
    """Function that returns the number of days between 2 dates """

    return abs((datetime.strptime(start_date, "%Y-%m-%d") - datetime.strptime(end_date, "%Y-%m-%d")).days)

def get_tickers():
    """Method that gets the stock symbols from companies listed in the S&P 500

    Return
    ------
    `tickers` : list
        S&P 500 company symbols
    """
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find_all('table')[0]  # Grab the first table

    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip('\n')
        tickers.append(ticker)

    return tickers

class Init():
    """Class that initializes global value for the module. It also use general method to initialize value.
     """

    def __init__(self):
        """Built-in method to inialize the global values for the module

        Attributes
        -----------
        `self.start.date` : str
            start date of the training period. Must be within the last year for the free version of FinHub. Format
            must be "YYYY-mm-dd"
        `self.end_date` : str
            end date of the training period. Format must be "YYYY-mm-dd"
        `self.ticker` : list
            tickers on which we want to perform the test. Can be one ticker in form of a list as well as a list
            of tickers like the s&p 500.
        `self.db_name` : str
            name of the sqlite3 database
        `self.dir_path` : str
            directory where the data are saved. It takes into account the `self.start_date` and `self.end_date`
        `self.start_date_` : datetime object
            same thing as `start_date` but as a datetime object
        `self.end_date_` : datetime object
            same thing as `start_date` but as a datetime object
        """

        #initialize value here
        self.start_date = "2021-12-15"
        self.end_date = "2022-12-13"
        self.tickers = ['AMZN']

        self.db_name = 'financial_data'
        self.dir_path = os.path.dirname(os.path.realpath('/content')) + '/output/' + self.start_date + '_' + \
                        self.end_date + '/'
        Path(self.dir_path).mkdir(parents=True, exist_ok=True) #create new path if it doesn't exist
        self.start_date_ = datetime.strptime(self.start_date, "%Y-%m-%d")  #datetime object
        self.end_date_ = datetime.strptime(self.end_date, "%Y-%m-%d")    #datetime object
        self.delta_date = abs((self.end_date_ - self.start_date_).days) #number of days between 2 dates


        try:
            self.start_date_ > self.end_date_
        except:
            print("'start_date' is after 'end_date'")

        t = (datetime.now()- relativedelta(years=1))
        d= datetime.strptime(self.start_date, "%Y-%m-%d")

        if (datetime.strptime(self.start_date, "%Y-%m-%d") <= (datetime.now()- relativedelta(years=1))) :
            raise Exception("'start_date' is older than 1 year. It doesn't work with the free version of FinHub")

class FinnHub():
    """Class to make API calls to FinnHub"""

    def __init__(self,start_date,end_date,start_date_,end_date_,tickers,dir_path,db_name):
        """ Class constructor

        Parameters
        ----------
        `start_date` : str
            Start date of the request. Must be within 1 year from now for must request
            with the free version of FinHub
        `end_date` : str
            End date of the request.
        `start_date_` : datetime object
            Same thing as `start_date` but as a datetime object
        `end_date_` : datetime object
             Same thing as `start_date` but as a datetime object
        `ticker` : str
            Ticker symbol
        `db_name` : str
            Name of the sqlite database
        `dir_path` : str
            Directory  where our data will be stored

        Attributes
        ----------
        `self.max_call` : int
            maximum api calls per minute for the finhub API
        `self.time_sleep` : int
            seconds to sleep before making a new API call. Default is 60 seconds as the maximum number of API calls is
            per minute
        `self.nb_request` : int
            nb of request made so far. Set to 0 in constructor `__init__` as we may loop through ticker
            and want to avoid the variable to reset to 0 when exiting the wrapper `iterate_day()` (which could generate
            an error)
        `self.finhub_key` : str
            finhub unique API key. Get yours here : https://finnhub.io/
        `self.db_name : str
            default file name for the sql database
        """

        #Initialize attributes values here
        self.max_call = 60
        self.time_sleep = 60
        self.nb_request = 0
        self.finhub_key = 'cecuohaad3i8too9m2v0cecuohaad3i8too9m2vg'
        self.news_header = ['category', 'datetime','headline','id','image','related','source','summary','url']
        self.start_date = start_date
        self.end_date = end_date
        self.tickers = tickers
        self.ticker_request = tickers #different value because ticker like 'ALL' (All State) can generate error in SQLite
                                    #database
        self.dir_path = dir_path
        self.db_name = db_name
        self.js_data = []

        self.start_date_ = start_date_ #datetime object
        self.end_date_ = end_date_ #datetime object

        #call the methods to access historical financial headlines
        #tickers = get_tickers() #get_tickers is to get tickers from all the companies listedin the s&p 500

        for ticker_ in self.tickers:
            self.js_data.clear()
            self.ticker = ticker_ + '_'
            self.ticker_request = ticker_
            self.req_new()
            self.create_table()
            self.clean_table()
            self.lang_review()

    def init_sql(func):
        """ Decorator that open the sql database, save it and close it. The operation are between the opening and
        saving of the file"""

        def wrapper_(self):
            conn_ = sqlite3.connect(self.dir_path + self.db_name + '.db')
            c = conn_.cursor()
            func(self,conn_,c)
            conn_.commit()
            conn_.close()
        return wrapper_

    @init_sql
    def clean_table(self,conn_,c):
        """Method that clean the database using sqlite3

        Parameters
        ----------
        `conn_` : database object
            Connection object that represents the database
        `c` : database object
            Cursor object
        """

        #remove NULL entry (row) from headline column
        c.execute(f" DELETE FROM {self.ticker} WHERE {self.news_header[2]} IS NULL OR "
                  f"trim({self.news_header[2]}) = '';")
        # remove NULL value from datetime
        c.execute(f" DELETE FROM {self.ticker} WHERE {self.news_header[1]} IS NULL OR "
                  f"trim({self.news_header[1]}) = '';")

        #removes duplicate entries (row)
        c.execute(f" DELETE FROM {self.ticker} WHERE rowid NOT IN (select MIN(rowid)"
                  f"FROM {self.ticker} GROUP BY {self.news_header[2]})")

    @init_sql
    def create_table(self,conn_,c):
        """ Method that creates a table in SQLite database. It creates the table  in `self.dir_path` and write
        the data in it

        Parameters
        ----------
        `conn_` : database object
            Connection object that represents the database
        `c` : database object
            Cursor object
        """

        #create table if it does not exist
        c.execute(f'drop table if exists {self.ticker}')
        conn_.commit()
        c.execute(f"CREATE TABLE IF NOT EXISTS {self.ticker} ({self.news_header[0]})")
        conn_.commit()

        #add columns to the table if the columns don't exist
        for header_ in range(len(self.news_header)-1):
            c.execute(f"alter table {self.ticker} add column '%s' " % self.news_header[header_+1])
            conn_.commit()

        iteration = 0
        for data_ in self.js_data:
            iteration +=1
            try :
                c.execute(f'insert into {self.ticker} values (?,?,?,?,?,?,?,?,?)',[data_[self.news_header[0]],
                          data_[self.news_header[1]],data_[self.news_header[2]],data_[self.news_header[3]],
                        data_[self.news_header[4]],data_[self.news_header[5]],data_[self.news_header[6]],
                          data_[self.news_header[7]],data_[self.news_header[8]]])
            except:
                print(f"Error at the {iteration}th ieration")

            conn_.commit()

    def iterate_day(func):
        """ Decorator that makes the API call on FinHub each days between the `self.start_date`
        and `self.end_date` """

        def wrapper_(self):
            delta_date_ = delta_date(self.start_date,self.end_date)
            date_ = self.start_date
            date_obj = self.start_date_

            for item in range(delta_date_ + 1):
                self.nb_request +=1
                func(self,date_)
                date_obj = date_obj + relativedelta(days=1)
                date_  = date_obj.strftime("%Y-%m-%d")
                if self.nb_request == (self.max_call-1):
                    time.sleep(self.time_sleep)
                    self.nb_request=0
        return wrapper_

    @init_sql
    def lang_review(self,conn_,c):
        """ Methods that delete non-english entries based on the 'headline' column in a SQLlite3 db

        Parameters
        ----------
        `conn_` : database object
            Connection object that represents the database
        `c` : database object
            Cursor object
        """

        list_ = []
        c.execute(f" SELECT {self.news_header[2]} FROM {self.ticker}")

        #check for non-english headlines
        for item_ in c:
            if detect(item_[0]) != 'en':
                list_.append(item_[0])

        #delete non-english entries (rows)
            query = f"DELETE FROM {self.ticker} where {self.news_header[2]} in ({','.join(['?']*len(list_))})"
            c.execute(query, list_)

    @iterate_day
    def req_new(self,date_):
        """ Method that makes news request(s) to the Finnhub API"""

        request_ = requests.get('https://finnhub.io/api/v1/company-news?symbol=' + self.ticker_request + '&from=' +
                                date_ + '&to=' + date_ + '&token=' + self.finhub_key)
        self.js_data += request_.json()




In [ ]:
init_ = Init()

finhub = FinnHub(start_date=init_.start_date, end_date=init_.end_date,start_date_=init_.start_date_ ,
                end_date_ =init_.end_date_, tickers=init_.tickers, dir_path =init_.dir_path,db_name=init_.db_name)

In [ ]:
import pandas as pd

In [ ]:
req1 = requests.get('https://finnhub.io/api/v1/company-news?symbol=' + 'AMZN' + '&from=' +
                                "2021-12-10" + '&to=' + "2021-12-19" + '&token=' + 'ced14sqad3i8too9n7qgced14sqad3i8too9n7r0')

In [ ]:
req1_json=req1.json()

In [ ]:
test_df1 = pd.DataFrame.from_dict(req1_json)

In [ ]:
test_df1

,category,datetime,headline,id,image,related,source,summary,url
0,company,1639956660,Future Group stocks surge 20% after India susp...,82877198,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Shares of India's Future Group companies surge...,https://finnhub.io/api/news?id=0cf83767ce6c8be...
1,company,1639956300,Alibaba climate tide can lift valuation boat,82678246,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,HONG KONG (Reuters Breakingviews) - Alibab...,https://finnhub.io/api/news?id=b2659f0ab6c2489...
2,company,1639954654,The Endless Volatility Is Pushing Holders To S...,82672399,,AMZN,SeekingAlpha,,https://finnhub.io/api/news?id=2d64f23a30273d9...
3,company,1639950300,Here's what's coming to Amazon's Prime Video i...,84339942,https://images.mktw.net/im-455191/social,AMZN,MarketWatch,Ben Affleck in 'The Tender Bar' and a new 'Hot...,https://finnhub.io/api/news?id=eb879ff008aca58...
4,company,1639923255,SCHG: Don't Give Up On Big-Tech Growth Stocks,82514282,https://static.seekingalpha.com/cdn/s3/uploads...,AMZN,SeekingAlpha,All the market chatter about rising inflation ...,https://finnhub.io/api/news?id=37a788ab1e25d9f...
5,company,1639920540,'Spider-Man: No Way Home' Rocks The US Box Off...,82850943,https://cdn.benzinga.com/files/imagecache/og_i...,AMZN,Benzinga,Sony Pictures (NYSE: SONY) scored a grand slam...,https://finnhub.io/api/news?id=39207211afb7688...
6,company,1639904400,"A Tech To-Do List, if You’re the Family IT Dep...",83398599,https://images.wsj.net/im-454706/social,AMZN,DowJones,"While you’re home for the holidays, sit your l...",https://finnhub.io/api/news?id=9a67967d502addf...
7,company,1639902528,Analysts' Top 10 And Bottom 10 Large Cap Picks...,82423110,https://static.seekingalpha.com/cdn/s3/uploads...,AMZN,SeekingAlpha,This article examines analyst ratings and 2022...,https://finnhub.io/api/news?id=c672bd328d3fa06...
8,company,1639900800,3 Takeaways From Wayfair's Brick-and-Mortar Plans,82421522,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Wayfair (NYSE: W) has made billions by being o...,https://finnhub.io/api/news?id=357a808e00c25dc...
9,company,1639900200,3 Underrated Warren Buffett Stocks That Are Sm...,82421544,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Warren Buffett is known for being an investor ...,https://finnhub.io/api/news?id=5bae146fc11a8e4...


In [ ]:
dt = datetime.fromtimestamp(1639893300)
dt

datetime.datetime(2021, 12, 19, 5, 55)

In [ ]:
req2 = requests.get('https://finnhub.io/api/v1/company-news?symbol=' + 'AMZN' + '&from=' +
                                "2021-12-15" + '&to=' + "2021-12-19" + '&token=' + 'ced14sqad3i8too9n7qgced14sqad3i8too9n7r0')

In [ ]:
req2_json=req2.json()

In [ ]:
test_df2 = pd.DataFrame.from_dict(req2_json)

In [ ]:
test_df2

,category,datetime,headline,id,image,related,source,summary,url
0,company,1639956660,Future Group stocks surge 20% after India susp...,82877198,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Shares of India's Future Group companies surge...,https://finnhub.io/api/news?id=0cf83767ce6c8be...
1,company,1639956300,Alibaba climate tide can lift valuation boat,82678246,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,HONG KONG (Reuters Breakingviews) - Alibab...,https://finnhub.io/api/news?id=b2659f0ab6c2489...
2,company,1639954654,The Endless Volatility Is Pushing Holders To S...,82672399,,AMZN,SeekingAlpha,,https://finnhub.io/api/news?id=2d64f23a30273d9...
3,company,1639950300,Here's what's coming to Amazon's Prime Video i...,84339942,https://images.mktw.net/im-455191/social,AMZN,MarketWatch,Ben Affleck in 'The Tender Bar' and a new 'Hot...,https://finnhub.io/api/news?id=eb879ff008aca58...
4,company,1639923255,SCHG: Don't Give Up On Big-Tech Growth Stocks,82514282,https://static.seekingalpha.com/cdn/s3/uploads...,AMZN,SeekingAlpha,All the market chatter about rising inflation ...,https://finnhub.io/api/news?id=37a788ab1e25d9f...
5,company,1639920540,'Spider-Man: No Way Home' Rocks The US Box Off...,82850943,https://cdn.benzinga.com/files/imagecache/og_i...,AMZN,Benzinga,Sony Pictures (NYSE: SONY) scored a grand slam...,https://finnhub.io/api/news?id=39207211afb7688...
6,company,1639904400,"A Tech To-Do List, if You’re the Family IT Dep...",83398599,https://images.wsj.net/im-454706/social,AMZN,DowJones,"While you’re home for the holidays, sit your l...",https://finnhub.io/api/news?id=9a67967d502addf...
7,company,1639902528,Analysts' Top 10 And Bottom 10 Large Cap Picks...,82423110,https://static.seekingalpha.com/cdn/s3/uploads...,AMZN,SeekingAlpha,This article examines analyst ratings and 2022...,https://finnhub.io/api/news?id=c672bd328d3fa06...
8,company,1639900800,3 Takeaways From Wayfair's Brick-and-Mortar Plans,82421522,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Wayfair (NYSE: W) has made billions by being o...,https://finnhub.io/api/news?id=357a808e00c25dc...
9,company,1639900200,3 Underrated Warren Buffett Stocks That Are Sm...,82421544,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Warren Buffett is known for being an investor ...,https://finnhub.io/api/news?id=5bae146fc11a8e4...


In [ ]:
dt = datetime.fromtimestamp(1639893300)
dt

datetime.datetime(2021, 12, 19, 5, 55)

In [ ]:
df2 = pd.concat([test_df1, test_df2], ignore_index=True, sort=False)

In [ ]:
df = pd.concat([df, df2], ignore_index=True, sort=False)

In [ ]:
df=df.drop_duplicates()

In [ ]:
df

,category,datetime,headline,id,image,related,source,summary,url
0,company,1671015459,FedEx Stock Offers Tremendous Value,117515521,https://static.seekingalpha.com/cdn/s3/uploads...,AMZN,SeekingAlpha,FedEx operates in 220 countries. This establis...,https://finnhub.io/api/news?id=1171e24c1b7e809...
1,company,1670987400,US Stocks: Making Sense Of 2023 Earnings Forec...,117510905,https://static.seekingalpha.com/cdn/s3/uploads...,AMZN,SeekingAlpha,Thereâs a battle brewing over 2023 earnings ...,https://finnhub.io/api/news?id=d2263cb8b9bd615...
2,company,1670978280,Amazon Offers Some Members a New Deal,117504887,,AMZN,Yahoo,If you're in a devilish mood and want to stop ...,https://finnhub.io/api/news?id=4e1654270ca5a8a...
3,company,1670969889,Amazon to shut down Kennesaw facility,117504888,,AMZN,Yahoo,As Amazon reportedly looks to cut thousands of...,https://finnhub.io/api/news?id=9c2376084532f94...
4,company,1670968018,"Amazon Inspire may show customers ‘content, pr...",117504889,https://s.yimg.com/ny/api/res/1.2/09J2dEBW4PEY...,AMZN,Yahoo,Head of Amazon Shopping Oliver Messenger joins...,https://finnhub.io/api/news?id=2f5e2fadf01a82b...
...,...,...,...,...,...,...,...,...,...
13686,company,1639902528,Analysts' Top 10 And Bottom 10 Large Cap Picks...,82423110,https://static.seekingalpha.com/cdn/s3/uploads...,AMZN,SeekingAlpha,This article examines analyst ratings and 2022...,https://finnhub.io/api/news?id=c672bd328d3fa06...
13687,company,1639900800,3 Takeaways From Wayfair's Brick-and-Mortar Plans,82421522,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Wayfair (NYSE: W) has made billions by being o...,https://finnhub.io/api/news?id=357a808e00c25dc...
13688,company,1639900200,3 Underrated Warren Buffett Stocks That Are Sm...,82421544,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Warren Buffett is known for being an investor ...,https://finnhub.io/api/news?id=5bae146fc11a8e4...
13689,company,1639898100,3 Favorite Growth Stocks for 2022,82421549,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,"After enduring a year's worth of abuse, many g...",https://finnhub.io/api/news?id=a1f6ede2ad5961e...


In [ ]:
df.to_csv('/content/AMZNnewsdata.csv')

In [ ]:
dt = datetime.fromtimestamp(1639902528)
dt

datetime.datetime(2021, 12, 19, 8, 28, 48)

In [ ]:
dt.strftime('%Y-%m-%d %H:%M')

'2021-12-19 08:28'

In [ ]:
dff2=df.copy()

In [ ]:
dates=[]

In [ ]:
for i in range(0,len(dff2['datetime'])):
  dates.append(datetime.fromtimestamp(dff2['datetime'].iloc[i]).strftime('%Y-%m-%d'))


In [ ]:
dff2['date']=dates

In [ ]:
dff2

,category,datetime,headline,id,image,related,source,summary,url,date
0,company,1671015459,FedEx Stock Offers Tremendous Value,117515521,https://static.seekingalpha.com/cdn/s3/uploads...,AMZN,SeekingAlpha,FedEx operates in 220 countries. This establis...,https://finnhub.io/api/news?id=1171e24c1b7e809...,2022-12-14
1,company,1670987400,US Stocks: Making Sense Of 2023 Earnings Forec...,117510905,https://static.seekingalpha.com/cdn/s3/uploads...,AMZN,SeekingAlpha,Thereâs a battle brewing over 2023 earnings ...,https://finnhub.io/api/news?id=d2263cb8b9bd615...,2022-12-14
2,company,1670978280,Amazon Offers Some Members a New Deal,117504887,,AMZN,Yahoo,If you're in a devilish mood and want to stop ...,https://finnhub.io/api/news?id=4e1654270ca5a8a...,2022-12-14
3,company,1670969889,Amazon to shut down Kennesaw facility,117504888,,AMZN,Yahoo,As Amazon reportedly looks to cut thousands of...,https://finnhub.io/api/news?id=9c2376084532f94...,2022-12-13
4,company,1670968018,"Amazon Inspire may show customers ‘content, pr...",117504889,https://s.yimg.com/ny/api/res/1.2/09J2dEBW4PEY...,AMZN,Yahoo,Head of Amazon Shopping Oliver Messenger joins...,https://finnhub.io/api/news?id=2f5e2fadf01a82b...,2022-12-13
...,...,...,...,...,...,...,...,...,...,...
13675,company,1639900800,3 Takeaways From Wayfair's Brick-and-Mortar Plans,82421522,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Wayfair (NYSE: W) has made billions by being o...,https://finnhub.io/api/news?id=357a808e00c25dc...,2021-12-19
13676,company,1639900200,3 Underrated Warren Buffett Stocks That Are Sm...,82421544,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Warren Buffett is known for being an investor ...,https://finnhub.io/api/news?id=5bae146fc11a8e4...,2021-12-19
13677,company,1639898100,3 Favorite Growth Stocks for 2022,82421549,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,"After enduring a year's worth of abuse, many g...",https://finnhub.io/api/news?id=a1f6ede2ad5961e...,2021-12-19
13678,company,1639893300,"5 Unstoppable Trends to Invest $5,000 In for 2022",82421567,https://www.nasdaq.com/sites/acquia.prod/files...,AMZN,Nasdaq,Think about personal computers in the early 19...,https://finnhub.io/api/news?id=7e04c3910263df5...,2021-12-19


In [ ]:
dff2.to_csv('AMZNNews.csv')